In [1]:
import csv
import os
import string
from collections import Counter
from collections import defaultdict

import nltk

In [2]:
spanish_stopwords = set()
with open('data/spanish_stopwords.txt') as f:
    for line in f:
        word = line.strip()
        spanish_stopwords.add(word)

In [3]:
%%time

alltokens = defaultdict(int)
count = 0
rows = []

path = 'data/recipes/elbulli/'
for folder in os.listdir(path):
    for filename in os.listdir(path + folder):
        with open(path + folder + '/' + filename) as f:
            text = ''
            for line in f:
                line = line.strip()
                if line.startswith('&titol') or \
                    line.startswith('&ingredientselaboracio') or \
                    line.startswith('&descripcioelaboracio') or \
                    line.startswith('&acabatipresentacio') or \
                    line.startswith('&titolelaboracio'):
                        equals_index = line.index('=')
                        text += line[equals_index + 1:].lower() + ' - '
            tokens = nltk.word_tokenize(text)
            for token in tokens:
                token = token.strip(string.punctuation)
                alltokens[token] += 1

CPU times: user 11.2 s, sys: 24 ms, total: 11.2 s
Wall time: 11.5 s


In [4]:
histogram = {}
for token in alltokens:
    conditions = [
        len(token) > 2,
        token not in spanish_stopwords,
        token not in string.punctuation,
        any(map(lambda x: x.isalpha(), token)),
#         alltokens[token] > 1,
    ]
    if all(conditions):
        histogram[token] = alltokens[token]

In [5]:
counter = Counter(histogram)

In [6]:
len(counter)

6653

In [7]:
counter.most_common(10)

[('agua', 6653),
 ('aceite', 3943),
 ('poner', 3872),
 ('sal', 3436),
 ('guardar', 2809),
 ('dejar', 2658),
 ('nevera', 2521),
 ('cortar', 2302),
 ('azúcar', 2275),
 ('gelatina', 2261)]

In [8]:
with open('data/elbulli_words.csv', 'w') as f:
    writer = csv.writer(
        f,
        delimiter=',',
        quotechar='"',
        quoting=csv.QUOTE_MINIMAL
    )
    for word, freq in counter.most_common():
        writer.writerow([word, freq])

In [9]:
# # Sort spanish_stopwords.txt

# spanish_stopwords = set()
# with open('data/spanish_stopwords.txt') as f:
#     for line in f:
#         word = line.strip()
#         spanish_stopwords.add(word)

# spanish_stopwords = sorted(spanish_stopwords)
# with open('data/spanish_stopwords.txt', 'w') as f:
#     for word in spanish_stopwords:
#         f.write(word + '\n')